<a href="https://colab.research.google.com/github/CygnusST3RN/FHE.0/blob/ankita_ml/FHE_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [12]:
def read_dataset(url,col,scale):
  df=pd.read_csv(url)
  df.drop("Unnamed: 32", axis=1, inplace=True)
  X = df.iloc[:, col+1:]
  col_name=df.columns[col]
  y = df[col_name]
  for i in range (0,len(df)):
    if y[i]=='M':
      y[i]=1
    else:
      y[i]=0
  X_train, X_test, y_train, y_test = train_test_split(
  X,y , random_state=104,test_size=0.25, shuffle=True)
  X_train=np.asarray(X_train).astype('float32')
  X_test=np.asarray(X_test).astype('float32')
  y_train=np.asarray(y_train).astype('float32')
  y_test=np.asarray(y_test).astype('float32')
  scaler = MinMaxScaler(feature_range=(0, scale))
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)
  print(X_test.shape)
  return X_train, X_test, y_train, y_test


In [13]:
X_train, X_test, y_train, y_test = read_dataset('data.csv',1,1)

(143, 30)


<ipython-input-12-581388e63b9f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=1
<ipython-input-12-581388e63b9f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=0


In [ ]:
df=pd.read_csv('data.csv')
dataset=df.to_numpy()
df.drop("Unnamed: 32", axis=1, inplace=True)
dataset.shape

(569, 33)

In [ ]:
X = df.iloc[:, 2:]
y = df["diagnosis"]
for i in range (0,569):
    if y[i]=='M':
        y[i]=1
    else:
        y[i]=0
print(df["diagnosis"])


<ipython-input-88-0ed101fed6dd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=1
<ipython-input-88-0ed101fed6dd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[i]=0


0      1
1      1
2      1
3      1
4      1
      ..
564    1
565    1
566    1
567    1
568    0
Name: diagnosis, Length: 569, dtype: object


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
  X,y , random_state=104,test_size=0.25, shuffle=True)

In [ ]:
X_train=np.asarray(X_train).astype('float32')
X_test=np.asarray(X_test).astype('float32')
y_train=np.asarray(y_train).astype('float32')
y_test=np.asarray(y_test).astype('float32')

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 100))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train=np.round(X_train)
X_test=np.round(X_test)

In [ ]:
X_train[0]

array([25.,  9., 24., 14., 45., 14.,  9., 18., 49., 20.,  5.,  7.,  4.,
        2., 22.,  7.,  4., 24., 16.,  4., 20., 10., 18.,  9., 46., 11.,
       10., 32., 25., 11.], dtype=float32)

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(30,)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=16)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Epoch 1/10
27/27 [==============================] - 1s 4ms/step - loss: 0.6950 - accuracy: 0.3732
Epoch 2/10
27/27 [==============================] - 0s 4ms/step - loss: 0.6789 - accuracy: 0.4014
Epoch 3/10
27/27 [==============================] - 0s 4ms/step - loss: 0.6656 - accuracy: 0.5305
Epoch 4/10
27/27 [==============================] - 0s 4ms/step - loss: 0.6542 - accuracy: 0.6667
Epoch 5/10
27/27 [==============================] - 0s 4ms/step - loss: 0.6428 - accuracy: 0.7512
Epoch 6/10
27/27 [==============================] - 0s 4ms/step - loss: 0.6326 - accuracy: 0.8333
Epoch 7/10
27/27 [==============================] - 0s 4ms/step - loss: 0.6224 - accuracy: 0.8568
Epoch 8/10
27/27 [==============================] - 0s 4ms/step - loss: 0.6131 - accuracy: 0.8873
Epoch 9/10
27/27 [==============================] - 0s 4ms/step - loss: 0.6041 - accuracy: 0.8873
Epoch 10/10
27/27 [==============================] - 0s 4ms/step - loss: 0.5952 - accuracy: 0.8991
Test Accuracy: 87.4

In [15]:
jloss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy*100:.2f}%")

Test Accuracy: 87.41%


In [ ]:
print(model.predict(np.array([X_test[3]])))
print(y_test[3])

1/1 [==============================] - 0s 20ms/step
[[2.867092e-09]]
0.0


In [ ]:
X_test[3]

array([ 6., 24.,  5.,  2., 30., 11.,  4.,  3., 38., 31.,  2., 13.,  2.,
        0., 25., 10.,  4., 11., 25.,  5.,  4., 26.,  3.,  1., 40., 12.,
        5.,  9., 30., 16.], dtype=float32)

In [ ]:
layer=model.layers[0]
print(len(model.layers))
w=layer.get_weights()[0][:,0] # weights
b=layer.get_weights()[1]
res=0
for i in range(len(w)):
  res+=X_test[3][i]*np.round(w[i]*100)
res+=np.round(b*100)
res/100

1


array([-19.48], dtype=float32)

In [ ]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_6 (Quantize  (None, 30)                3         
 Layer)                                                          
                                                                 
 quant_dense_7 (QuantizeWra  (None, 1)                 36        
 pperV2)                                                         
                                                                 
Total params: 39 (156.00 Byte)
Trainable params: 31 (124.00 Byte)
Non-trainable params: 8 (32.00 Byte)
_________________________________________________________________


In [ ]:
q_aware_model.fit(X_train, y_train, epochs=30, batch_size=16)

Epoch 1/30
27/27 [==============================] - 1s 3ms/step - loss: nan - accuracy: 0.6455
Epoch 2/30
27/27 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.6338
Epoch 3/30
27/27 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.6338
Epoch 4/30
27/27 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.6338
Epoch 5/30
27/27 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.6338
Epoch 6/30
27/27 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.6338
Epoch 7/30
27/27 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.6338
Epoch 8/30
27/27 [==============================] - 0s 4ms/step - loss: nan - accuracy: 0.6338
Epoch 9/30
27/27 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.6338
Epoch 10/30
27/27 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.6338
Epoch 11/30
27/27 [==============================

In [ ]:
-